- ufunc

In [238]:
n=9
idxs=np.arange(n**2)
grid=np.arange(n**2).reshape(n,n)
block=np.stack([grid[i*3:(i+1)*3,j*3:(j+1)*3].flatten() for i in range(3) for j in range(3)])
unitlist=np.concatenate([grid, grid.T,block])

def get_units(s):
    return unitlist[np.isin(unitlist,s).sum(-1).astype(bool)]
units=np.array([get_units(s) for s in range(n**2)])

def get_peers(s):
    a=np.unique(get_units(s))
    idx=a!=s
    return a[idx].tolist()
peers=np.array([get_peers(s) for s in idxs])

In [239]:
a="4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
b= '003020600900305001001806400008102900700000008006708200002609500800203009005010300'

In [240]:
def myint(x):
    if x==".":
        return -1
    else:
        return int(x)-1
    
def grid_values(grid):
    vals = np.array([myint(c) for c in grid if c in (digits+1).astype(str) or c in '0.'])
    assert len(vals) == 81
    mask=np.isin(vals,digits)
    return idxs[mask], vals[mask]

In [241]:
digits = np.arange(n)
def parse_grid(grid):
    values=np.full((n**2,n),True)
    uassign=np.frompyfunc(lambda x, y:assign(values, x,y), 2, 1)
    uelim_peers=np.frompyfunc(lambda x, y:elim_peers(values, x,y), 2, 1)
    def process(s,d):
        uassign(s, d)
        uelim_peers(s,d)

    uprocess=np.frompyfunc(process, 2, 1)
    done=np.full_like(idxs, False)
    cand=grid_values(grid)
    
    while True:
        uprocess(*cand)
        cand=np.where(values*(values.sum(-1) == 1)[:,None]*(done[:,None]==False))
        if len(cand[0]) == 0:
            break
        done[cand[0]]=True
    return values

In [242]:
def assign(values, s, d):
    d2=digits[values[s]]
    values[s, d2]=False
    values[s, d]=True

In [243]:
def elim_peers(values, s, d):
    values[peers[s],d]=False

In [244]:
def display(values):
    "valuesを2次元のテキスト形式で表示する。"
    width = 1+values.sum(-1).max()
    line = '+'.join(['-'*(width*3)]*3)
    
    def tostr(x):
        return "".join((digits[x]+1).astype(str)).center(width)
    
    x=np.apply_along_axis(tostr, -1, values[grid])
    for r in range(n):
        print(*list(x[r,c]+('|' if c == 2 or c == 5 else '') for c in range(n)))
        if r == 2 or r==5: print(line)

In [245]:
digits = np.arange(n)
def _solve(grid): return _search(parse_grid(grid))
def _search(values):
    
    if values is False:
        return False ## 前の時点で失敗している
    if (values.sum(-1)==1).prod():
        return values ## 解けた!
    
    x=values.sum(-1)
    x[x==1] *= 10
    s=x.argmin()
    v=digits[values[s]]
    
    def process(v, values):
        values=values.copy()
        uassign=np.frompyfunc(lambda x:assign(values, s, x), 1, 1)
        uelim_peers=np.frompyfunc(lambda x:elim_peers(values, s,x), 1, 1)
        done=np.full_like(idxs, False)
        while True:
            uassign(v)
            uelim_peers(v)

            # check
            if values.sum(-1).prod() == 0:
                return False
            
            cand=np.where(values*(values.sum(-1) == 1)[:,None]*(done[:,None]==False))
            if len(cand[0]) == 0:
                break
            done[cand[0]]=True
        return values
    
    return some(_search(process(vv, values)) for vv in v)

def some(seq):
    for e in seq:
        if e is not False: return e
    return False

In [246]:
digits = np.arange(n)
def solve(grid): return search(parse_grid(grid))
def search(values):
    
    if values is False:
        return False ## 前の時点で失敗している
    if (values.sum(-1)==1).prod():
        return values ## 解けた!
    
    x=values.sum(-1)
    x[x==1] *= 10
    s=x.argmin()
    v=digits[values[s]]
    
    def process(v, values):
        values=values.copy()
        uassign=np.frompyfunc(lambda x:assign(values, s, x), 1, 1)
        uelim_peers=np.frompyfunc(lambda x:elim_peers(values, s,x), 1, 1)
        
        uassign(v)
        uelim_peers(v)
        # check
        if values.sum(-1).prod() == 0:
            return False
        return values
    
    return some(search(process(vv, values)) for vv in v)

def some(seq):
    for e in seq:
        if e is not False: return e
    return False

In [235]:
%%timeit
(solve(a))

7.67 ms ± 84.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [226]:
%%timeit
(solve(b))

4.61 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [229]:
%%timeit
(_solve(a))

15.6 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [230]:
%%timeit
(_solve(b))

4.65 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
